# 01.use_Rasa

## 1.1 Use Rasa NLU

In [2]:
import logging
import pprint
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer
from rasa_nlu.model import Interpreter
from rasa_nlu.test import run_evaluation

In [3]:
logfile = 'nlu_model.log'
def train_nlu(data_path, configs, model_path):
    logging.basicConfig(filename=logfile, level=logging.DEBUG)
    training_data = load_data(data_path)
    trainer = Trainer(config.load(configs))
    trainer.train(training_data)
    model_directory = trainer.persist(model_path, project_name='current', fixed_model_name='nlu')
    run_evaluation(data_path, model_directory)
    s
train_nlu('./data/nlu.md', 'nlu_config.yml', './models')

In [23]:
nlu_path = './models/current/nlu'
interpreter = Interpreter.load(nlu_path)

In [34]:
# pprint.pprint(interpreter.parse("Share some latest news around the world?"))
# pprint.pprint(interpreter.parse("What is going on in technology?"))
# pprint.pprint(interpreter.parse("What is going on in education?"))
user_text = "hello"
data_analyst = interpreter.parse(user_text)
intent = data_analyst['intent']['name']
default_entity = ''
default_entity_value = ''
if data_analyst['entities'] != []:
    default_entity = data_analyst['entities'][0]['entity']
    default_entity_value = data_analyst['entities'][0]['value']
entity = default_entity
entity_value = default_entity_value
print('Intent: ', intent)
print('Entity: ', entity)
print('Entity value:', entity_value)

Intent:  greet
Entity:  
Entity value: 


## 1.2 User Rasa Core

In [ ]:
import logging
from rasa_core import config
from rasa_core import utils
from rasa_core.channels.slack import SlackInput
from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter
from rasa_core.utils import EndpointConfig


logfile = 'dialogue_model.log'


def train_core(domain_file, model_path, training_data_file, policy_config):
    logging.basicConfig(filename=logfile, level=logging.DEBUG)
    agent = Agent(domain_file, policies=config.load(policy_config))
    training_data = agent.load_data(training_data_file)
    agent.train(training_data)
    agent.persist(model_path)
    return agent


def run_core(core_model_path, nlu_model_path, action_endpoint_url, slack_token):
    logging.basicConfig(filename=logfile, level=logging.DEBUG)
    nlu_interpreter = RasaNLUInterpreter(nlu_model_path)
    action_endpoint = EndpointConfig(url=action_endpoint_url)
    agent = Agent.load(core_model_path, interpreter=nlu_interpreter, action_endpoint=action_endpoint)
    input_channel = SlackInput(slack_token)
    agent.handle_channels([input_channel], 5004, serve_forever=True)

    print("Your bot is ready to talk! Type your messages here or send 'stop'")
    while True:
        a = input()
        if a == 'stop':
            break
        responses = agent.handle_text(a)
        for response in responses:
            print(response["text"])
    return agent


if __name__ == '__main__':
    
    run_core('./models/dialogue', './models/current/nlu',
             actionConfig["action_endpoint"]["url"], slackConfig["slack"]["slack_token"])

In [ ]:
actionConfig = utils.read_yaml_file('endpoints.yml')
slackConfig = utils.read_yaml_file('credentials.yml')
train_core('domain.yml', './models/dialogue', './data/stories.md', 'policy.yml')